In [ ]:
import json
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from folium.plugins import HeatMap
import folium as fl
import datetime
sns.set(style="whitegrid")
%matplotlib inline
from ipywidgets import *
from ipywidgets import interact, interactive, fixed
from IPython.display import display
#from IPython.html import widgets
import ipywidgets as widgets


In [ ]:
widgets.IntSlider()

In [ ]:
battles = pd.DataFrame([json.loads(line) for line in open("../datasets/battle-features-0.json")])

In [ ]:
m = fl.Map()
def plotHeatMap(battles,m):
    coord_df = battles[["latitude", "longitude"]].dropna()
    coords = [[lat, long] for lat, long in zip(coord_df["latitude"], coord_df["longitude"])]
    HeatMap(coords).add_to(m)
    return m

In [ ]:
btp = battles[['start_date','longitude','latitude','dates_bc','end_date']]
btp = btp[pd.notnull(btp['start_date'])]
btp = btp[pd.notnull(btp['longitude'])]
btp = btp[pd.notnull(btp['latitude'])]

btpbc=btp[btp.dates_bc==True]
btpbc['year'] = [-datetime.datetime.strptime(date, "%Y-%m-%d").year for date in btpbc['start_date']]
btpnbc=btp[btp.dates_bc==False]
btpnbc['year'] = [datetime.datetime.strptime(date, "%Y-%m-%d").year for date in btpnbc['start_date']]
btp=btpbc.append(btpnbc)

X=[datetime.datetime.strptime(date, "%Y-%m-%d") for date in btp['start_date']]
fig, ax = plt.subplots(figsize=(20,1))
ax.scatter(X, [1]*len(X),
           marker='|', s=100)

ax.yaxis.set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_ticks_position('bottom')

ax.get_yaxis().set_ticklabels([])
day = pd.to_timedelta("1", unit='D')
plt.xlim(X[0] - day, X[-1] + day)
plt.show()

In [ ]:
bins = [-2500,-1000,-500,-250,0,250,500,750,1000,1250,1500,1750,2000,2250]
bins_labels=[-2500,-1000,-500,-250,0,250,500,750,1000,1250,1500,1750,2000]
btpyear=btp.groupby(pd.cut(btp['year'], bins=bins)).year
years_count = btpyear.count()
years_count.plot(kind='bar')


In [ ]:
lengthbins = [-1,5,10,15,50,100,150,200,500,1000,2000,3000]
lengthbins_labels=[0,5,10,15,50,100,150,200,500,1000,2000]

days=br.groupby(pd.cut(br['length'], bins=lengthbins, labels=lengthbins_labels))['length']
days_count = days.count()
days_count.plot(kind='bar')

In [ ]:

btp = btp.groupby(pd.cut(btp['year'],bins=bins, labels=bins_labels))[['start_date','end_date','latitude','longitude']]
  
btp1 = btp.get_group(-2500)
btp2 = btp.get_group(-1000)
btp3 = btp.get_group(-500)
btp4 = btp.get_group(-250)
btp5 = btp.get_group(0)
btp6 = btp.get_group(250)
btp7 = btp.get_group(500)
btp8 = btp.get_group(750)
btp9 = btp.get_group(1000)
btp10 = btp.get_group(1250)
btp11 = btp.get_group(1500)
btp12 = btp.get_group(1750)
btp13 = btp.get_group(2000)


In [ ]:
m = fl.Map()
m= plotHeatMap(btp1,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp2,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp3,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp4,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp5,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp6,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp7,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp8,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp9,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp10,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp11,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp12,m)
m

In [ ]:
m = fl.Map()
m= plotHeatMap(btp13,m)
m

In [ ]:
clusters=[btp5,btp6,btp7,btp8,btp9,btp10,btp11,btp12,btp13]
clustersbc=[btp1,btp2,btp3,btp4] #none of these clusters have end_date values so all battles BC are considered to have lasted only on a day
lengthbins = [-1,5,10,15,50,100,150,200,500,1000,2000,3000]
lengthbins_labels=[0,5,10,15,50,100,150,200,500,1000,2000]
 
newCluster=[]
for b in clusters:
    b['end_date']=b['end_date'].fillna(b['start_date'])
    b['start_date'] = b['start_date'].astype('datetime64[ns]')
    b['end_date'] = b['end_date'].astype('datetime64[ns]')
    length=[]
    b=b.reset_index()
    for i in range(0,len(b['start_date'])):
        #print(pd.Timedelta(btp1.end_date[i]-btp1.start_date[i]))
        length+=[pd.Timedelta(b.end_date[i]-b.start_date[i])]
    b['length'] = length
    b['length']=b['length'].astype('timedelta64[D]')
    newCluster+=[b]
    

In [ ]:
def plotDurationByPeriod(b):
    days=b.groupby(pd.cut(b['length'], bins=lengthbins, labels=lengthbins_labels))['length']
    days_count = days.count()
    plt.figure(i)
    days_count.plot(kind='bar')
    plt.title('duration of battles in cluster')

interact(plotDurationByPeriod, b)

In [ ]:
plotDurationByPeriod(b=btp5)

In [ ]:
btp5

In [ ]:

btp = battles[['start_date','longitude','latitude','dates_bc','end_date']]
btp = btp[pd.notnull(btp['start_date'])]
btp = btp[pd.notnull(btp['longitude'])]
btp = btp[pd.notnull(btp['latitude'])]

btpbc=btp[btp.dates_bc==True]
btpbc['year'] = [-datetime.datetime.strptime(date, "%Y-%m-%d").year for date in btpbc['start_date']]
btpnbc=btp[btp.dates_bc==False]
btpnbc['year'] = [datetime.datetime.strptime(date, "%Y-%m-%d").year for date in btpnbc['start_date']]
btp=btpbc.append(btpnbc)

bins = [-2500,-1000,-500,-250,0,250,500,750,1000,1250,1500,1750,2000,2250]
bins_labels=[-2500,-1000,-500,-250,0,250,500,750,1000,1250,1500,1750,2000]
lengthbins = [-1,5,10,15,50,100,150,200,500,1000,2000,3000]
lengthbins_labels=[0,5,10,15,50,100,150,200,500,1000,2000]
btp = btp.groupby(pd.cut(btp['year'],bins=bins, labels=bins_labels))[['start_date','end_date','latitude','longitude']]
  
def function(b):
    btpnow = btp.get_group(b)
    btpnow['end_date']=btpnow['end_date'].fillna(btpnow['start_date'])
    btpnow['start_date'] = btpnow['start_date'].astype('datetime64[ns]')
    btpnow['end_date'] = btpnow['end_date'].astype('datetime64[ns]')
    length=[]
    btpnow=btpnow.reset_index()
    for i in range(0,len(btpnow['start_date'])):
        #print(pd.Timedelta(btp1.end_date[i]-btp1.start_date[i]))
        length+=[pd.Timedelta(btpnow.end_date[i]-btpnow.start_date[i])]
    btpnow['length'] = length
    btpnow['length']=btpnow['length'].astype('timedelta64[D]')
    days=btpnow.groupby(pd.cut(btpnow['length'], bins=lengthbins, labels=lengthbins_labels))['length']
    days_count = days.count()
    plt.figure(i)
    days_count.plot(kind='bar')
    plt.title('duration of battles in cluster')


In [ ]:
interact(function,b=(0,2000,250) )

In [ ]:
def f(x):
    print(x)
widget = interact(f,x=10)
display(widget)